# Load and inspect the provided preprocessed ISPU dataset

In [16]:
import pandas as pd
import numpy as np

df = pd.read_csv("./datasets/ispu_preprocessed_3.csv")

# Basic inspection outputs
print("Shape:", df.shape)
print("\nColumns:")
print(df.columns.tolist())

print("\nInfo:")
print(df.info())

print("\nMissing values per column:")
print(df.isna().sum().sort_values(ascending=False).head(20))

print("\nSample rows:")
print(df.head())

df = df.rename(columns={
    "pm_sepuluh": "pm10",
    "pm_duakomalima": "pm25",
    "sulfur_dioksida": "so2",
    "karbon_monoksida": "co",
    "ozon": "o3",
    "nitrogen_dioksida": "no2"
})

pollutant_cols = [
    "pm10",
    "pm25",
    "so2",
    "co",
    "o3",
    "no2"
]

# pollutant_cols = [
#     "pm_sepuluh",
#     "pm_duakomalima",
#     "sulfur_dioksida",
#     "karbon_monoksida",
#     "ozon",
#     "nitrogen_dioksida"
# ]

print("\nMissing values (pollutants):")
print(df[pollutant_cols].isna().sum())

print("\nBasic statistics (pollutants):")
print(df[pollutant_cols].describe())

# Check unique categories and stations
print("\nUnique kategori:", df['kategori'].unique())
print("Unique stasiun:", df['stasiun'].unique())

Shape: (15356, 12)

Columns:
['periode_data', 'tanggal', 'stasiun', 'pm_sepuluh', 'pm_duakomalima', 'sulfur_dioksida', 'karbon_monoksida', 'ozon', 'nitrogen_dioksida', 'max', 'parameter_pencemar_kritis', 'kategori']

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15356 entries, 0 to 15355
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   periode_data               15356 non-null  int64  
 1   tanggal                    15356 non-null  object 
 2   stasiun                    15356 non-null  object 
 3   pm_sepuluh                 14648 non-null  float64
 4   pm_duakomalima             6579 non-null   float64
 5   sulfur_dioksida            15057 non-null  float64
 6   karbon_monoksida           15150 non-null  float64
 7   ozon                       15013 non-null  float64
 8   nitrogen_dioksida          15046 non-null  float64
 9   max                        15356 non-null  floa

## Trying BRITS  

In [ ]:
!pip install pypots

In [17]:
df["tanggal"] = pd.to_datetime(df["tanggal"])
df = df.sort_values(["stasiun", "tanggal"])

sequences = []
station_lengths = []

for station in df["stasiun"].unique():
    seq = df[df["stasiun"] == station][pollutant_cols].values
    sequences.append(seq)
    station_lengths.append(len(seq))

X = np.array(sequences, dtype=object)


In [19]:
max_len = max(len(seq) for seq in sequences)
n_features = len(pollutant_cols)

X = np.full((len(sequences), max_len, n_features), np.nan)

for i, seq in enumerate(sequences):
    X[i, :len(seq), :] = seq


In [22]:
from pypots.imputation import BRITS

brits = BRITS(
    n_steps=max_len,
    n_features=n_features,
    rnn_hidden_size=64,
    batch_size=4,
    epochs=10,
    device="cpu"
)

train_set = {"X": X}
brits.fit(train_set)

X_imputed = brits.impute({"X": X})

2026-02-06 22:21:15 [INFO]: Using the given device: cpu
2026-02-06 22:21:15 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2026-02-06 22:21:15 [INFO]: Using customized MAE as the training loss function.
2026-02-06 22:21:15 [INFO]: Using customized MSE as the validation metric function.
2026-02-06 22:21:15 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 41,936
2026-02-06 22:21:19 [INFO]: Epoch 001 - training loss (MAE): 77.9196
2026-02-06 22:21:23 [INFO]: Epoch 002 - training loss (MAE): 73.1785
2026-02-06 22:21:27 [INFO]: Epoch 003 - training loss (MAE): 68.1332
2026-02-06 22:21:32 [INFO]: Epoch 004 - training loss (MAE): 71.4050
2026-02-06 22:21:36 [INFO]: Epoch 005 - training loss (MAE): 76.4520
2026-02-06 22:21:40 [INFO]: Epoch 006 - training loss (MAE): 70.7308
2026-02-06 22:21:45 [INFO]: Epoch 007 - training loss (MAE): 73.4964
2026-02-06 22:21:49 [INFO]: Epoch 008 - training loss (MAE): 71.127

In [23]:
imputed_list = []

for i, station in enumerate(df["stasiun"].unique()):
    seq_len = len(df[df["stasiun"] == station])
    
    temp = df[df["stasiun"] == station].copy()
    temp[pollutant_cols] = X_imputed[i, :seq_len, :]
    
    imputed_list.append(temp)

df_imputed = pd.concat(imputed_list).sort_values(["stasiun","tanggal"])


In [24]:
df_imputed[pollutant_cols].isna().sum()


pm10    0
pm25    0
so2     0
co      0
o3      0
no2     0
dtype: int64

In [27]:
df_imputed.pivot_table(index="tanggal", columns="stasiun", values="pm25").corr()


stasiun,DKI1,DKI2,DKI3,DKI4,DKI5
stasiun,,,,,
DKI1,1.000000,0.971644,0.905143,0.906454,0.771657
DKI2,0.971644,1.000000,0.895011,0.906838,0.786701
DKI3,0.905143,0.895011,1.000000,0.840255,0.715102
DKI4,0.906454,0.906838,0.840255,1.000000,0.682180
DKI5,0.771657,0.786701,0.715102,0.682180,1.000000


In [ ]:
# df_imputed.to_csv("ispu_imputed_BRITS.csv", index=False)

In [6]:
df["tanggal"] = pd.to_datetime(df["tanggal"])
df = df.sort_values("tanggal")

In [7]:
df["year"] = df["tanggal"].dt.year
df["month"] = df["tanggal"].dt.month
df["day"] = df["tanggal"].dt.day
df["dayofweek"] = df["tanggal"].dt.dayofweek
df["weekofyear"] = df["tanggal"].dt.isocalendar().week.astype(int)
df["quarter"] = df["tanggal"].dt.quarter
df["is_weekend"] = (df["dayofweek"] >= 5).astype(int)

In [8]:
df

,periode_data,tanggal,stasiun,pm10,pm25,so2,co,o3,no2,max,parameter_pencemar_kritis,kategori,year,month,day,dayofweek,weekofyear,quarter,is_weekend
0,201001,2010-01-01,DKI1,60.0,51.0,4.0,73.0,27.0,14.0,73.0,CO,SEDANG,2010,1,1,4,53,1,0
13411,201001,2010-01-01,DKI5,57.0,62.0,39.0,35.0,197.0,11.0,0.0,NaN,BAIK,2010,1,1,4,53,1,0
6375,201001,2010-01-01,DKI3,69.0,89.0,0.0,19.0,80.0,6.0,0.0,NaN,BAIK,2010,1,1,4,53,1,0
9719,201001,2010-01-01,DKI4,27.0,101.0,8.0,22.0,45.0,10.0,0.0,NaN,BAIK,2010,1,1,4,53,1,0
2894,201001,2010-01-01,DKI2,27.0,60.0,8.0,22.0,45.0,10.0,0.0,NaN,BAIK,2010,1,1,4,53,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2893,202508,2025-08-31,DKI1,42.0,70.0,29.0,12.0,15.0,24.0,70.0,PM25,SEDANG,2025,8,31,6,35,3,1
9718,202508,2025-08-31,DKI3,28.0,60.0,53.0,8.0,19.0,39.0,60.0,PM25,SEDANG,2025,8,31,6,35,3,1
13410,202508,2025-08-31,DKI4,47.0,59.0,27.0,10.0,18.0,17.0,59.0,PM25,SEDANG,2025,8,31,6,35,3,1
6374,202508,2025-08-31,DKI2,76.0,72.0,45.0,16.0,21.0,16.0,72.0,PM25,SEDANG,2025,8,31,6,35,3,1


In [9]:
time_features = [
    "year",
    "month",
    "day",
    "dayofweek",
    "weekofyear",
    "quarter",
    "is_weekend",
    "stasiun"
]


In [10]:
df[time_features + ["pm10","pm25","so2","co","o3","no2"]].head()

,year,month,day,dayofweek,weekofyear,quarter,is_weekend,stasiun,pm10,pm25,so2,co,o3,no2
0,2010,1,1,4,53,1,0,DKI1,60.0,51.0,4.0,73.0,27.0,14.0
13411,2010,1,1,4,53,1,0,DKI5,57.0,62.0,39.0,35.0,197.0,11.0
6375,2010,1,1,4,53,1,0,DKI3,69.0,89.0,0.0,19.0,80.0,6.0
9719,2010,1,1,4,53,1,0,DKI4,27.0,101.0,8.0,22.0,45.0,10.0
2894,2010,1,1,4,53,1,0,DKI2,27.0,60.0,8.0,22.0,45.0,10.0


In [11]:
split_date = df["tanggal"].quantile(0.8)

train = df[df["tanggal"] <= split_date]
test  = df[df["tanggal"] > split_date]


In [13]:
import catboost

In [18]:
!pip install prophet


Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 9.7 MB/s  0:00:01m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [prophet]m4/5 [prophet]]

[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [19]:
from prophet import Prophet

def forecast_pollutant(df, station, pollutant, future_dates):
    df_st = df[df["stasiun"] == station][["tanggal", pollutant]].copy()
    df_st.columns = ["ds", "y"]

    model = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=False
    )

    model.fit(df_st)

    future = pd.DataFrame({"ds": future_dates})
    forecast = model.predict(future)

    return forecast[["ds", "yhat"]]


In [20]:
split_date = "2024-01-01"

df_train_ts = df[df["tanggal"] < split_date].copy()
df_test_ts  = df[df["tanggal"] >= split_date].copy()

In [21]:
df_forecasted = df_test_ts[["tanggal","stasiun"]].copy()

for pol in pollutant_cols:
    print(f"Forecasting {pol}...")
    
    all_preds = []
    
    for station in df_test_ts["stasiun"].unique():
        dates = df_test_ts[df_test_ts["stasiun"] == station]["tanggal"]
        
        fc = forecast_pollutant(df_train_ts, station, pol, dates)
        fc["stasiun"] = station
        
        all_preds.append(fc)
    
    all_preds = pd.concat(all_preds)
    
    df_forecasted = df_forecasted.merge(
        all_preds.rename(columns={"ds":"tanggal", "yhat":pol}),
        on=["tanggal","stasiun"],
        how="left"
    )

19:25:58 - cmdstanpy - INFO - Chain [1] start processing


Forecasting pm10...


19:26:00 - cmdstanpy - INFO - Chain [1] done processing
/Users/gracewhyn/Library/Python/3.9/lib/python/site-packages/prophet/forecaster.py:1421: RuntimeWarning: divide by zero encountered in matmul
  comp = np.matmul(X, beta_c.transpose())
/Users/gracewhyn/Library/Python/3.9/lib/python/site-packages/prophet/forecaster.py:1421: RuntimeWarning: overflow encountered in matmul
  comp = np.matmul(X, beta_c.transpose())
/Users/gracewhyn/Library/Python/3.9/lib/python/site-packages/prophet/forecaster.py:1421: RuntimeWarning: invalid value encountered in matmul
  comp = np.matmul(X, beta_c.transpose())
/Users/gracewhyn/Library/Python/3.9/lib/python/site-packages/prophet/forecaster.py:1565: RuntimeWarning: divide by zero encountered in matmul
  Xb_a = np.matmul(seasonal_features.values,
/Users/gracewhyn/Library/Python/3.9/lib/python/site-packages/prophet/forecaster.py:1565: RuntimeWarning: overflow encountered in matmul
  Xb_a = np.matmul(seasonal_features.values,
/Users/gracewhyn/Library/Python

Forecasting pm25...


19:26:02 - cmdstanpy - INFO - Chain [1] start processing
19:26:02 - cmdstanpy - INFO - Chain [1] done processing
/Users/gracewhyn/Library/Python/3.9/lib/python/site-packages/prophet/forecaster.py:1421: RuntimeWarning: divide by zero encountered in matmul
  comp = np.matmul(X, beta_c.transpose())
/Users/gracewhyn/Library/Python/3.9/lib/python/site-packages/prophet/forecaster.py:1421: RuntimeWarning: overflow encountered in matmul
  comp = np.matmul(X, beta_c.transpose())
/Users/gracewhyn/Library/Python/3.9/lib/python/site-packages/prophet/forecaster.py:1421: RuntimeWarning: invalid value encountered in matmul
  comp = np.matmul(X, beta_c.transpose())
/Users/gracewhyn/Library/Python/3.9/lib/python/site-packages/prophet/forecaster.py:1565: RuntimeWarning: divide by zero encountered in matmul
  Xb_a = np.matmul(seasonal_features.values,
/Users/gracewhyn/Library/Python/3.9/lib/python/site-packages/prophet/forecaster.py:1565: RuntimeWarning: overflow encountered in matmul
  Xb_a = np.matmul(

...

19:26:05 - cmdstanpy - INFO - Chain [1] start processing
19:26:05 - cmdstanpy - INFO - Chain [1] done processing
/Users/gracewhyn/Library/Python/3.9/lib/python/site-packages/prophet/forecaster.py:1421: RuntimeWarning: divide by zero encountered in matmul
  comp = np.matmul(X, beta_c.transpose())
/Users/gracewhyn/Library/Python/3.9/lib/python/site-packages/prophet/forecaster.py:1421: RuntimeWarning: overflow encountered in matmul
  comp = np.matmul(X, beta_c.transpose())
/Users/gracewhyn/Library/Python/3.9/lib/python/site-packages/prophet/forecaster.py:1421: RuntimeWarning: invalid value encountered in matmul
  comp = np.matmul(X, beta_c.transpose())
/Users/gracewhyn/Library/Python/3.9/lib/python/site-packages/prophet/forecaster.py:1565: RuntimeWarning: divide by zero encountered in matmul
  Xb_a = np.matmul(seasonal_features.values,
/Users/gracewhyn/Library/Python/3.9/lib/python/site-packages/prophet/forecaster.py:1565: RuntimeWarning: overflow encountered in matmul
  Xb_a = np.matmul(

Forecasting o3...


19:26:06 - cmdstanpy - INFO - Chain [1] done processing
/Users/gracewhyn/Library/Python/3.9/lib/python/site-packages/prophet/forecaster.py:1421: RuntimeWarning: divide by zero encountered in matmul
  comp = np.matmul(X, beta_c.transpose())
/Users/gracewhyn/Library/Python/3.9/lib/python/site-packages/prophet/forecaster.py:1421: RuntimeWarning: overflow encountered in matmul
  comp = np.matmul(X, beta_c.transpose())
/Users/gracewhyn/Library/Python/3.9/lib/python/site-packages/prophet/forecaster.py:1421: RuntimeWarning: invalid value encountered in matmul
  comp = np.matmul(X, beta_c.transpose())
/Users/gracewhyn/Library/Python/3.9/lib/python/site-packages/prophet/forecaster.py:1565: RuntimeWarning: divide by zero encountered in matmul
  Xb_a = np.matmul(seasonal_features.values,
/Users/gracewhyn/Library/Python/3.9/lib/python/site-packages/prophet/forecaster.py:1565: RuntimeWarning: overflow encountered in matmul
  Xb_a = np.matmul(seasonal_features.values,
/Users/gracewhyn/Library/Python

Forecasting no2...


/Users/gracewhyn/Library/Python/3.9/lib/python/site-packages/prophet/forecaster.py:1421: RuntimeWarning: divide by zero encountered in matmul
  comp = np.matmul(X, beta_c.transpose())
/Users/gracewhyn/Library/Python/3.9/lib/python/site-packages/prophet/forecaster.py:1421: RuntimeWarning: overflow encountered in matmul
  comp = np.matmul(X, beta_c.transpose())
/Users/gracewhyn/Library/Python/3.9/lib/python/site-packages/prophet/forecaster.py:1421: RuntimeWarning: invalid value encountered in matmul
  comp = np.matmul(X, beta_c.transpose())
/Users/gracewhyn/Library/Python/3.9/lib/python/site-packages/prophet/forecaster.py:1565: RuntimeWarning: divide by zero encountered in matmul
  Xb_a = np.matmul(seasonal_features.values,
/Users/gracewhyn/Library/Python/3.9/lib/python/site-packages/prophet/forecaster.py:1565: RuntimeWarning: overflow encountered in matmul
  Xb_a = np.matmul(seasonal_features.values,
/Users/gracewhyn/Library/Python/3.9/lib/python/site-packages/prophet/forecaster.py:1565

In [23]:
from catboost import CatBoostClassifier

model = CatBoostClassifier()
model.load_model("catboost_kategori.cbm")

feature_cols = model.feature_names_

In [24]:
df_forecasted

,tanggal,stasiun,pm10,pm25,so2,co,o3,no2
0,2024-01-01,DKI1,39.463289,73.001936,50.613535,4.922721,18.285463,18.636982
1,2024-01-01,DKI2,37.742404,71.060779,32.557821,6.530574,10.007299,21.088254
2,2024-01-01,DKI4,46.478064,90.779084,32.038545,18.851930,10.452966,11.887249
3,2024-01-01,DKI5,28.662926,58.911396,22.974011,13.636418,20.894101,10.806930
4,2024-01-01,DKI3,39.575833,67.094354,50.639917,7.503768,-1.792920,7.445826
...,...,...,...,...,...,...,...,...
3040,2025-08-31,DKI1,61.588365,90.808861,62.837959,4.837714,34.815012,26.750065
3041,2025-08-31,DKI3,63.757047,69.098223,62.079669,4.414555,17.237722,4.569194
3042,2025-08-31,DKI4,74.655770,104.043526,27.908867,26.040610,16.047195,12.647919
3043,2025-08-31,DKI2,65.457672,80.003796,33.947012,3.464339,0.978447,23.899147


In [25]:
df_forecasted["year"] = df_test_ts["year"].values
df_forecasted["month"] = df_test_ts["month"].values
df_forecasted["day"] = df_test_ts["day"].values
df_forecasted["dayofweek"] = df_test_ts["dayofweek"].values
df_forecasted["weekofyear"] = df_test_ts["weekofyear"].values
df_forecasted["quarter"] = df_test_ts["quarter"].values
df_forecasted["is_weekend"] = df_test_ts["is_weekend"].values


In [26]:
df_forecasted

,tanggal,stasiun,pm10,pm25,so2,co,o3,no2,year,month,day,dayofweek,weekofyear,quarter,is_weekend
0,2024-01-01,DKI1,39.463289,73.001936,50.613535,4.922721,18.285463,18.636982,2024,1,1,0,1,1,0
1,2024-01-01,DKI2,37.742404,71.060779,32.557821,6.530574,10.007299,21.088254,2024,1,1,0,1,1,0
2,2024-01-01,DKI4,46.478064,90.779084,32.038545,18.851930,10.452966,11.887249,2024,1,1,0,1,1,0
3,2024-01-01,DKI5,28.662926,58.911396,22.974011,13.636418,20.894101,10.806930,2024,1,1,0,1,1,0
4,2024-01-01,DKI3,39.575833,67.094354,50.639917,7.503768,-1.792920,7.445826,2024,1,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3040,2025-08-31,DKI1,61.588365,90.808861,62.837959,4.837714,34.815012,26.750065,2025,8,31,6,35,3,1
3041,2025-08-31,DKI3,63.757047,69.098223,62.079669,4.414555,17.237722,4.569194,2025,8,31,6,35,3,1
3042,2025-08-31,DKI4,74.655770,104.043526,27.908867,26.040610,16.047195,12.647919,2025,8,31,6,35,3,1
3043,2025-08-31,DKI2,65.457672,80.003796,33.947012,3.464339,0.978447,23.899147,2025,8,31,6,35,3,1


In [27]:
X_fake_test = df_forecasted[feature_cols]
y_pred = model.predict(X_fake_test)

from sklearn.metrics import classification_report

y_true = df_test_ts["kategori"]

print(classification_report(y_true, y_pred))


              precision    recall  f1-score   support

        BAIK       0.47      0.21      0.29       380
      SEDANG       0.79      0.88      0.83      2343
 TIDAK SEHAT       0.23      0.18      0.20       322

    accuracy                           0.72      3045
   macro avg       0.50      0.42      0.44      3045
weighted avg       0.69      0.72      0.70      3045



In [28]:
sample = pd.read_csv("sample_submission.csv")
sample.head()


,id,category
0,2025-09-01_DKI1,NaN
1,2025-09-01_DKI2,NaN
2,2025-09-01_DKI3,NaN
3,2025-09-01_DKI4,NaN
4,2025-09-01_DKI5,NaN


In [29]:
sample[["tanggal", "stasiun_kode"]] = sample["id"].str.split("_", expand=True)

sample["tanggal"] = pd.to_datetime(sample["tanggal"])

In [30]:
sample

,id,category,tanggal,stasiun_kode
0,2025-09-01_DKI1,NaN,2025-09-01,DKI1
1,2025-09-01_DKI2,NaN,2025-09-01,DKI2
2,2025-09-01_DKI3,NaN,2025-09-01,DKI3
3,2025-09-01_DKI4,NaN,2025-09-01,DKI4
4,2025-09-01_DKI5,NaN,2025-09-01,DKI5
...,...,...,...,...
450,2025-11-30_DKI1,NaN,2025-11-30,DKI1
451,2025-11-30_DKI2,NaN,2025-11-30,DKI2
452,2025-11-30_DKI3,NaN,2025-11-30,DKI3
453,2025-11-30_DKI4,NaN,2025-11-30,DKI4


In [31]:
from prophet import Prophet

def forecast_station(df_hist, station, target_dates):
    df_s = df_hist[df_hist["stasiun"] == station].copy()
    df_s = df_s.sort_values("tanggal")

    forecasts = []

    for pol in pollutant_cols:
        df_p = df_s[["tanggal", pol]].dropna()
        df_p.columns = ["ds", "y"]

        m = Prophet()
        m.fit(df_p)

        future = pd.DataFrame({"ds": target_dates})
        fc = m.predict(future)[["ds", "yhat"]]
        fc.columns = ["tanggal", pol]

        forecasts.append(fc)

    # gabungkan semua polutan
    df_fc = forecasts[0]
    for f in forecasts[1:]:
        df_fc = df_fc.merge(f, on="tanggal")

    df_fc["stasiun"] = station
    return df_fc


In [34]:
all_forecasts = []

for station in sample["stasiun_kode"].unique():
    target_dates = sample[sample["stasiun_kode"] == station]["tanggal"].values
    df_fc = forecast_station(df, station, target_dates)
    all_forecasts.append(df_fc)

df_forecasted = pd.concat(all_forecasts, ignore_index=True)
df_forecasted.head()

20:09:26 - cmdstanpy - INFO - Chain [1] start processing
20:09:26 - cmdstanpy - INFO - Chain [1] done processing
/Users/gracewhyn/Library/Python/3.9/lib/python/site-packages/prophet/forecaster.py:1421: RuntimeWarning: divide by zero encountered in matmul
  comp = np.matmul(X, beta_c.transpose())
/Users/gracewhyn/Library/Python/3.9/lib/python/site-packages/prophet/forecaster.py:1421: RuntimeWarning: overflow encountered in matmul
  comp = np.matmul(X, beta_c.transpose())
/Users/gracewhyn/Library/Python/3.9/lib/python/site-packages/prophet/forecaster.py:1421: RuntimeWarning: invalid value encountered in matmul
  comp = np.matmul(X, beta_c.transpose())
/Users/gracewhyn/Library/Python/3.9/lib/python/site-packages/prophet/forecaster.py:1565: RuntimeWarning: divide by zero encountered in matmul
  Xb_a = np.matmul(seasonal_features.values,
/Users/gracewhyn/Library/Python/3.9/lib/python/site-packages/prophet/forecaster.py:1565: RuntimeWarning: overflow encountered in matmul
  Xb_a = np.matmul(

,tanggal,pm10,pm25,so2,co,o3,no2,stasiun
0,2025-09-01,53.272534,80.402058,26.327209,18.840674,19.592837,38.065682,DKI1
1,2025-09-02,54.270946,80.185300,27.015627,19.776844,18.503192,39.303561,DKI1
2,2025-09-03,54.876824,80.560976,27.179653,20.286136,19.727644,40.097976,DKI1
3,2025-09-04,54.993791,80.511244,27.426414,20.078263,20.057687,39.913029,DKI1
4,2025-09-05,55.402798,79.950646,27.137119,20.094219,20.934656,39.254798,DKI1


In [36]:
feature_cols = model.feature_names_
X_submit = df_forecasted[feature_cols]
sample["category"] = model.predict(X_submit).ravel()

In [39]:
feature_cols

['stasiun', 'pm10', 'pm25', 'so2', 'co', 'o3', 'no2']

In [38]:
submission = sample[["id", "category"]]
submission.to_csv("submission5_prophet.csv", index=False)

In [48]:
sample

,id,category,tanggal,stasiun_kode
0,2025-09-01_DKI1,SEDANG,2025-09-01,DKI1
1,2025-09-01_DKI2,SEDANG,2025-09-01,DKI2
2,2025-09-01_DKI3,SEDANG,2025-09-01,DKI3
3,2025-09-01_DKI4,SEDANG,2025-09-01,DKI4
4,2025-09-01_DKI5,SEDANG,2025-09-01,DKI5
...,...,...,...,...
450,2025-11-30_DKI1,SEDANG,2025-11-30,DKI1
451,2025-11-30_DKI2,SEDANG,2025-11-30,DKI2
452,2025-11-30_DKI3,SEDANG,2025-11-30,DKI3
453,2025-11-30_DKI4,SEDANG,2025-11-30,DKI4


In [ ]:
pollutant_cols = ["pm10","pm25","so2","co","o3","no2"]

med = df_filled.groupby("stasiun")[pollutant_cols].median().reset_index()
sample = sample.merge(med, on="stasiun", how="left")


In [44]:
feature_cols = model.feature_names_
X_submit = sample[feature_cols]

sample["category"] = model.predict(X_submit).ravel()

# feature_cols = model.feature_names_
# X_submit = sample[feature_cols]

# pred = model.predict(X_submit).ravel()

# sample["category"] = pred

## XGBoost

In [41]:
from xgboost import XGBClassifier
import pickle

model_xgb = XGBClassifier()
model_xgb.load_model("xgb_kategori.json")

# load encoder
le_station = pickle.load(open("le_station.pkl", "rb"))
le_target = pickle.load(open("le_target.pkl", "rb"))

# misal ini data hasil forecasting prophet (sample)
df_sample = df_forecasted.copy()


In [42]:
df_sample["stasiun"] = le_station.transform(df_sample["stasiun"])

In [43]:
X_submit = df_sample[feature_cols]
y_pred_num = model_xgb.predict(X_submit)

In [44]:
y_pred_label = le_target.inverse_transform(y_pred_num)
df_sample["kategori"] = y_pred_label


In [45]:
df_sample

,tanggal,pm10,pm25,so2,co,o3,no2,stasiun,kategori
0,2025-09-01,53.272534,80.402058,26.327209,18.840674,19.592837,38.065682,0,SEDANG
1,2025-09-02,54.270946,80.185300,27.015627,19.776844,18.503192,39.303561,0,SEDANG
2,2025-09-03,54.876824,80.560976,27.179653,20.286136,19.727644,40.097976,0,SEDANG
3,2025-09-04,54.993791,80.511244,27.426414,20.078263,20.057687,39.913029,0,SEDANG
4,2025-09-05,55.402798,79.950646,27.137119,20.094219,20.934656,39.254798,0,SEDANG
...,...,...,...,...,...,...,...,...,...
450,2025-11-26,29.008590,79.920267,31.965587,11.886022,28.227834,20.420846,4,SEDANG
451,2025-11-27,29.644199,79.184113,31.998058,11.947746,28.038554,20.232600,4,SEDANG
452,2025-11-28,29.204838,78.549243,31.738331,11.721516,25.994905,20.223026,4,SEDANG
453,2025-11-29,28.621046,77.426131,31.818083,11.561295,25.105164,19.361081,4,SEDANG


In [53]:
df_sample.to_csv("submission_notclean.csv", index=False)

In [50]:
sample["category"] = le_target.inverse_transform(y_pred_num)

In [51]:
sample

,id,category,tanggal,stasiun_kode
0,2025-09-01_DKI1,SEDANG,2025-09-01,DKI1
1,2025-09-01_DKI2,SEDANG,2025-09-01,DKI2
2,2025-09-01_DKI3,SEDANG,2025-09-01,DKI3
3,2025-09-01_DKI4,SEDANG,2025-09-01,DKI4
4,2025-09-01_DKI5,SEDANG,2025-09-01,DKI5
...,...,...,...,...
450,2025-11-30_DKI1,SEDANG,2025-11-30,DKI1
451,2025-11-30_DKI2,SEDANG,2025-11-30,DKI2
452,2025-11-30_DKI3,SEDANG,2025-11-30,DKI3
453,2025-11-30_DKI4,SEDANG,2025-11-30,DKI4


In [52]:
submission = sample[["id", "category"]]
submission.to_csv("submission6_xgb.csv", index=False)

In [46]:
print("\nUnique kategori:", df_sample['kategori'].unique())


Unique kategori: ['SEDANG' 'BAIK']


In [49]:
feature_cols = model_xgb.feature_names_
X_submit_xgb = df_sample[feature_cols]
sample["category"] = model_xgb.predict(X_submit).ravel()

AttributeError: 'XGBClassifier' object has no attribute 'feature_names_'

In [ ]:
submission = sample[["id", "category"]]
submission.to_csv("submission5_prophet.csv", index=False)